In [1]:
# The imports

import os
from enum import Enum
from dotenv import load_dotenv
from agents import Agent, Runner, trace, WebSearchTool, gen_trace_id, function_tool
from agents.mcp import MCPServerStdio
from pydantic import BaseModel, EmailStr, Field
from typing import List, Optional

from pypdf import PdfReader
from IPython.display import Markdown, display

from openai import OpenAI
import json
import asyncio
from contextlib import AsyncExitStack


In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
    
brave_api_key = os.getenv('BRAVE_API_KEY')
if brave_api_key:
    print(f"Brave API Key exists and begins {brave_api_key[:12]}")


OpenAI API Key exists and begins sk-proj-
Brave API Key exists and begins BSAOqnqej-yp


In [4]:
MODEL = "gpt-5-mini"
#MODEL = "gpt-4.1-mini"

In [5]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

print(fetch_tools)

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

In [6]:
[print(f"Name: {tool.name}\n\tDescription: {tool.description}\n") for tool in fetch_tools]

Name: fetch
	Description: Fetches a URL from the internet and optionally extracts its contents as markdown.

Although originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.



[None]

In [7]:
data_files_path = os.path.abspath(os.path.join(os.getcwd(), "../data"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", data_files_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()


In [8]:
len(file_tools)

14

In [9]:
for tool in file_tools:
    print(f"Name: {tool.name}\nDescription: {tool.description}\n")

Name: read_file
Description: Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.

Name: read_text_file
Description: Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates on the file as text regardless of extension. Only works within allowed directories.

Name: read_media_file
Description: Read an image or audio file. Returns the base64 encoded data and MIME type. Only works within allowed directories.

Name: read_multiple_files
Description: Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its pat

In [10]:
# Create a pydentic model for the resume: include the following fields:
# - name
# - email
# - phone
# - linkedin
# - github
# - list of skills
# - list of experiences
# - list of education
# - list of projects
# - list of certifications
# - list of publications
# - list of patents
# - summary of resume in 2-3 sentences
# - list of all the relevant keywords that can be used to search jobs online

class ExperienceLevel(str, Enum):
    ENTRY = "entry" # 0-2 years
    JUNIOR = "junior" # 2-5 years
    MID = "mid" # 5-10 years    
    SENIOR = "senior" # 10-15 years
    LEAD = "lead" # 15+ years
    EXECUTIVE = "executive" # 20+ years

class Experience(BaseModel):
    title: str = Field(..., description="Job title or position held")
    company: Optional[str] = Field(None, description="Name of the company or organization")
    start_date: Optional[str] = Field(None, description="Start date of the experience")
    end_date: Optional[str] = Field(None, description="End date of the experience")
    description: Optional[str] = Field(None, description="Brief description of responsibilities and achievements")

class Education(BaseModel):
    degree: str = Field(..., description="Degree or qualification obtained")
    institution: Optional[str] = Field(None, description="Name of the educational institution")
    start_date: Optional[str] = Field(None, description="Start date of the education")
    end_date: Optional[str] = Field(None, description="End date of the education")
    description: Optional[str] = Field(None, description="Brief description of coursework or achievements")

class Project(BaseModel):
    name: str = Field(..., description="Name of the project")
    description: Optional[str] = Field(None, description="Brief description of the project")
    link: Optional[str] = Field(None, description="URL or link to the project")

class Certification(BaseModel):
    name: str = Field(..., description="Name of the certification")
    issuer: Optional[str] = Field(None, description="Issuing organization or authority")
    date: Optional[str] = Field(None, description="Date the certification was obtained")

class Publication(BaseModel):
    title: str = Field(..., description="Title of the publication")
    publisher: Optional[str] = Field(None, description="Publisher or journal name")
    date: Optional[str] = Field(None, description="Date of publication")
    link: Optional[str] = Field(None, description="URL or link to the publication")

class Patent(BaseModel):
    title: str = Field(..., description="Title of the patent")
    number: Optional[str] = Field(None, description="Patent number")
    date: Optional[str] = Field(None, description="Date the patent was granted")
    description: Optional[str] = Field(None, description="Brief description of the patent")

class ResumeModel(BaseModel):
    name: str = Field(..., description="Full name of the candidate")
    email: EmailStr = Field(..., description="Email address of the candidate")
    phone: Optional[str] = Field(None, description="Phone number of the candidate")
    linkedin: Optional[str] = Field(None, description="LinkedIn profile URL")
    github: Optional[str] = Field(None, description="GitHub profile URL")
    skills: List[str] = Field(default_factory=list, description="List of skills")
    experiences: List[Experience] = Field(default_factory=list, description="List of professional experiences")
    education: List[Education] = Field(default_factory=list, description="List of educational qualifications")
    projects: List[Project] = Field(default_factory=list, description="List of projects")
    certifications: List[Certification] = Field(default_factory=list, description="List of certifications")
    publications: List[Publication] = Field(default_factory=list, description="List of publications")
    patents: List[Patent] = Field(default_factory=list, description="List of patents")
    experience_level: ExperienceLevel = Field(..., description="Experience level of the candidate based on number of years of experience")
    summary: Optional[str] = Field(None, description="Summary of the resume in 2-3 sentences")
    keywords: List[str] = Field(default_factory=list, description="List of relevant keywords for job search")
    target_company_profile: Optional[str] = Field(None, description="Profile/sector of the target company used for job search")


# Create a pydentic model for the job search results:
# - job title
# - company
# - location
# - description
# - link
# - apply link  
# - Hiring manager name
# - list of all the relevant keywords that matched the job posting

class JobPosting(BaseModel):
    title: str = Field(..., description="EXACT Job title or position held")
    company: Optional[str] = Field(None, description="EXACT Name of the company or organization")
    location: Optional[str] = Field(None, description="EXACT Location of the job posting")
    description: Optional[str] = Field(None, description="Brief description of the job posting")
    link: Optional[str] = Field(None, description="URL or link to the job posting. (CRITICAL - this must be the actual job description page)")
    apply_link: Optional[str] = Field(None, description="URL or link to the job application. (CRITICAL - this must be the actual application link)")
    posted_date: Optional[str] = Field(None, description="Date the job posting was posted")
    application_deadline: Optional[str] = Field(None, description="Date the job application deadline")
    hiring_manager_name: Optional[str] = Field(None, description="Name of the hiring manager")
    keywords: List[str] = Field(default_factory=list, description="List of relevant keywords for the job posting")
    rating: Optional[float] = Field(None, description="Rating of the job posting from 0 to 10 based on the candidate's profile")

class JobPostingList(BaseModel):
    job_postings: List[JobPosting] = Field(default_factory=list, description="List of job postings")

In [11]:
@function_tool
async def read_resume(resume_file: str) -> str:
    """
    Read the resume from the given file path and return the text.
    @param resume_file: str - The path to the candidate's resume file
    Returns:
        str - The text of the resume
    """
    
    print(f"Reading resume from:\n {resume_file}")
    
    resume = ""
    try:
        # Read the resume
        reader = PdfReader(resume_file)
        for page in reader.pages:
            text = page.extract_text()
            if text:
                resume += text
    except Exception as e:
        print(f"Error reading resume from {resume_file}: {e}")
        raise ValueError(f"Error reading resume from {resume_file}: {e}")
        
    print(f"resume from function_tool:\n {resume}")
    return resume

class ResumeProcessorAgent:
    """
    Agent to process the resume and return the text.
    @param model: str - The model to use for the agent
    """
    def __init__(self, candidate_name: str, model: str = MODEL):
        self.candidate_name = candidate_name
        self.model = model

    def get_system_prompt(self) -> str:
        return f"""
        You are an expert Resume Processing Agent and AI recruiting copilot specializing in transforming raw resume data into structured, actionable job search profiles. 
        Your expertise lies in accurately analyzing professional backgrounds and creating comprehensive candidate profiles optimized for modern job search algorithms and recruiting systems.

        ## PRIMARY OBJECTIVE
        Transform the candidate's resume into a structured ResumeModel that maximizes their job search effectiveness by creating targeted, 
        ATS-friendly profiles that highlight their strongest qualifications and market positioning.

        ## EXECUTION WORKFLOW
        ### Step 1: Resume Data Extraction
        - Use the read_resume tool to extract complete text from the provided resume file
        - Parse and analyze all sections: contact info, summary, experience, education, skills, certifications, projects, etc.
        - Identify any formatting issues or missing information that might affect analysis

        ### Step 2: Comprehensive Analysis
        - Conduct thorough analysis of:
            - Professional trajectory: Career progression, role evolution, industry focus
            - Core competencies: Technical skills, domain expertise, leadership capabilities
            - Achievements: Quantifiable results, awards, recognitions, impact metrics
            - Education & credentials: Degrees, certifications, relevant coursework
            - Market positioning: How the candidate competes in their field

        ### Step 3: Structured Data Generation
        - Generate a complete ResumeModel conforming to this JSON schema:
            {ResumeModel.model_json_schema()}

        ## DETAILED FIELD REQUIREMENTS
        ### SUMMARY (Critical for Initial Screening)
        - Create a compelling 5-7 bullet point executive summary that:
            - Opens with a strong positioning statement (e.g., "Senior Full-Stack Engineer with 8+ years building scalable web applications")
            - Quantifies key achievements with specific metrics (revenue impact, team size, project scale)
            - Highlights unique value propositions (rare skill combinations, industry expertise, leadership experience)
            - Emphasizes recent/relevant experience most heavily, if the candidate has less than 2 years of experience, then emphasize the education and certifications.
            - Uses action-oriented language that demonstrates impact and results
            - Targets hiring manager perspective - what would make them want to interview this candidate?

        ### Example Format:
        - Senior Software Engineer with 6+ years developing enterprise SaaS platforms serving 100K+ users
        - Led cross-functional teams of 8+ engineers, delivering 15+ major features ahead of schedule  
        - Expertise in React, Node.js, AWS, reducing system latency by 40% and improving user engagement by 25%
        - Strong background in fintech and healthcare domains with deep understanding of compliance requirements
        - Proven track record of mentoring junior developers and establishing engineering best practices
        
        ## KEYWORDS (Boolean Search Optimization)
        ### Generate 15-20 strategic keywords optimized for:
        - Technical Skills: Programming languages, frameworks, tools, platforms
        - Methodologies: Agile, DevOps, CI/CD, testing practices
        - Domain Expertise: Industry-specific knowledge, compliance, regulations
        - Role Types: Job titles, seniority levels, functional areas
        - Certifications: Professional credentials, vendor certifications
        - Soft Skills: Leadership, communication, project management

        #### Boolean Search Format Examples:
        - "bioinformatics" AND ("machine learning" OR "self-supervised" OR "AI") AND (Python OR R), 
        - "precision medicine" AND ("tissue microarray" OR "image analysis" OR pathology) AND (self-supervised OR deep learning), 
        - "genomics" AND ("variant calling" OR alignment OR "genomic data visualization") AND (Python OR R), 
        - "single cell" OR "spatial transcriptomics" AND (analysis OR pipeline) AND (Snakemake OR Nextflow), 
        - (pharmaceutical OR drug testing), "workflow management" AND (Snakemake OR Nextflow) AND (Docker OR containerization), 
        - "bioinformatics" AND (HPC OR "high performance computing") AND (Python OR R), 
        - "multi-omics" AND (integration OR "data analysis") AND (R OR Python) 

        ### Exclude: Company names, location names, personal information

        ## EXPERIENCE LEVEL (Precise Classification)
        Calculate total professional work experience using these rules:
        ### Counting Rules:
        - Include: Full-time roles, significant part-time roles (20+ hours/week), consulting/freelance
        - Exclude: Internships, fellowships, student work, volunteer work
        - Overlapping roles: Count the primary role period only
        - Career gaps: Subtract gaps longer than 6 months unless for education/family reasons

        ### Classification Tiers:
        - ENTRY: New graduates OR <2 years professional experience. Ignore the internship or fellowship experience.
        - JUNIOR: 2-5 years of professional experience
        - MID: 5-10 years of professional experience
        - SENIOR: 10-15 years of professional experience
        - LEAD: 15-20 years of professional experience
        - EXECUTIVE: 20+ years of professional experience OR C-level/VP titles

        ## TARGET COMPANY PROFILE
        Analyze the candidate's background to identify ideal company characteristics:
        - Company size: Startup (1-50), Scale-up (51-500), Mid-market (501-5000), Enterprise (5000+)
        - Industry sectors: Based on previous experience and skills
        - Company stage: Early-stage, growth-stage, mature, public company
        - Culture indicators: Remote-first, innovation-focused, enterprise-focused, etc.
        - Technology stack alignment: Companies using similar technologies

        ## QUALITY ASSURANCE RULES
        ### Accuracy Standards
        - No hallucination: Only include information explicitly stated or clearly inferrable from the resume
        - Prefer null values over guessed information when data is unclear
        - Verify consistency between different resume sections
        - Cross-reference dates to ensure timeline accuracy

        ### Completeness Checks
        - Ensure all required ResumeModel fields are populated
        - Flag any critical missing information that might hurt job search effectiveness
        - Validate that keywords align with actual experience and skills

        ### Optimization Guidelines
        - ATS-friendly formatting: Use standard terminology and keywords
        - Recruiter-focused language: Emphasize measurable achievements and business impact
        - Market-aware positioning: Position candidate competitively within their experience level
        - Search-optimized content: Include terms commonly used in job postings for target roles

        ## OUTPUT REQUIREMENTS
        Format: Return ONLY the complete JSON object conforming to ResumeModel schema - no additional text, explanations, or formatting.
        ### Validation: Ensure the output:
        - Passes JSON schema validation
        - Contains all required fields
        - Uses appropriate data types
        - Includes realistic and accurate information
        - Optimizes for job search effectiveness

        ### EXAMPLE PROCESSING APPROACH
        - Extract: Parse resume sections systematically
        - Analyze: Identify key themes, progressions, and strengths
        - Synthesize: Create compelling summary highlighting unique value
        - Optimize: Generate keywords and positioning for target market
        - Structure: Format into clean ResumeModel JSON output
        - Validate: Verify accuracy and completeness before returning

        NOTE: Your success is measured by how effectively the generated profile helps the candidate land
        """

    def get_resume_file_path(self) -> str:
        return f"../data/{self.candidate_name}.pdf"

    def get_user_prompt(self) -> str:
        return f"""
        Use the text extracted from the PDF resume using the `read_resume` tool to create a job profile for the candidate that can be used for job search.
        read_resume function_tool is available to use and takes a single argument resume_file which is the path to the candidate's resume file. 
        The resume file path is: {self.get_resume_file_path()}
        """

    def get_resume_processor_agent(self) -> Agent:
        """
        This function is used to profile the resume and create a job profile for the candidate that can be used for job search.
        It create an agent that uses the `read_resume` tool to read the resume and calls the OpenAI API to create a job profile for the candidate.
        @return: Agent - The agent that can be used to profile the resume
        """

        print("started profile_resume")
        openai = OpenAI()

        system_prompt = self.get_system_prompt()
        print(f"system_prompt:\n {system_prompt}")

        resume_processor_agent = Agent(
            model=MODEL,
            name="ResumeProcessorAgent",
            instructions=system_prompt,
            tools=[read_resume],
            output_type=ResumeModel
        )

        return resume_processor_agent

    async def run_with_trace(self):
        """
        This function is used to run the agent with trace.
        """
        profiled_resume = None
        trace_id = gen_trace_id()
        print(f"Trace@ https://platform.openai.com/api/traces/{trace_id}")
        
        with trace(trace_id) as tracer:
            profiled_resume = await Runner.run(self.get_resume_processor_agent(), self.get_user_prompt())

        return profiled_resume.final_output if profiled_resume else None

In [12]:
profiled_resume = await ResumeProcessorAgent("Saaniya Desai", MODEL).run_with_trace()

print(f"profiled_resume:\n {profiled_resume}")

Trace@ https://platform.openai.com/api/traces/trace_cf0e9c35aa4748c1b77b8b7f7efe1080
started profile_resume
system_prompt:
 
        You are an expert Resume Processing Agent and AI recruiting copilot specializing in transforming raw resume data into structured, actionable job search profiles. 
        Your expertise lies in accurately analyzing professional backgrounds and creating comprehensive candidate profiles optimized for modern job search algorithms and recruiting systems.

        ## PRIMARY OBJECTIVE
        Transform the candidate's resume into a structured ResumeModel that maximizes their job search effectiveness by creating targeted, 
        ATS-friendly profiles that highlight their strongest qualifications and market positioning.

        ## EXECUTION WORKFLOW
        ### Step 1: Resume Data Extraction
        - Use the read_resume tool to extract complete text from the provided resume file
        - Parse and analyze all sections: contact info, summary, experience, edu

In [17]:
print(profiled_resume)


name='Saaniya Desai' email='saaniya.desai@gmail.com' phone=None linkedin=None github='https://github.com/saaniyadesai' skills=['Python', 'R', 'SQL', 'Bash', 'Git', 'Docker', 'HPC environments', 'Variant calling', 'Alignment tools', 'Genomic data visualization', 'Single-cell transcriptomics', 'Spatial transcriptomics', 'Multi-omics integration', 'Machine learning', 'Self-supervised learning', 'Unsupervised learning', 'Snakemake', 'Nextflow', 'LC-MS / GC-MS / IC', 'Mass spectrometry'] experiences=[Experience(title='MSc Bioinformatics Project — The Hughes Group', company='University of Glasgow', start_date='May 2024', end_date='Sep 2025', description='Developed a machine learning model to validate viral integration sites in cancer genomes; analyzed large-scale viral integration datasets to identify host–virus interaction trends with potential therapeutic relevance.'), Experience(title='MSc Precision Medicine Project — The Le Quesne Group', company='University of Glasgow', start_date='May 

In [27]:
class JobSearcherAgent:
    """
    Agent to search for jobs based on the job profile.
    @param job_profile: ResumeModel - The job profile of the candidate
    @param model: str - The model to use for the agent
    """
    def __init__(self, job_profile: ResumeModel, model: str = MODEL):
        self.job_profile = job_profile
        self.model = model
        self.brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

    def get_system_prompt(self) -> str:
        return f"""
        You are an expert Job Search Agent specializing in finding active job opportunities that perfectly match a candidate's profile. 
        Your mission is to conduct comprehensive job searches across multiple platforms and extract detailed, actionable information about each opportunity.
        You are given a candidate's profile as a JSON object represented by the Pydantic model ResumeModel.
        Your task is to search AS MANY job search sites as possible using specified MCP servers for searching the web and 
        return a list of job postings as a JSON object represented by the Pydantic model JobPostingList.

        ## Core Responsibilities
        - Using Candidate's Profile, find the list of top 10 companies that the candidate would be most interested in working for. Search the job postings on their career pages.
        - Deep Job Search: Use all available MCP tools to search extensively across job boards, company websites, and professional platforms
        - Active Job Validation: Verify that job postings are current, active, and accepting applications
        - Detailed Information Extraction: Extract comprehensive job details including application URLs, requirements, and company information
        - Profile Matching: Ensure all found opportunities align with the candidate's experience level, skills

        ## Search Strategy 
        ### Primary Search Sources (Use MCP tools for each):
        - Look for jobs in North America and Canada and United Kingdom
        - Major Job Boards: LinkedIn, Indeed, Glassdoor, ZipRecruiter, Monster or similar job boards
        - Tech-Specific Platforms: Stack Overflow Jobs, AngelList, Dice, GitHub Jobs or similar platforms
        - Company Career Pages: Direct searches on target company websites. 
        - Professional Networks: Remote work platforms, industry-specific job boards
        - Government/Public Sector: USAJobs, state government portals (if relevant)

        ### Search Methodology:
        - Keyword-Based Search: Use the candidate's summary for initial broad searches
        - Title-Specific Search: Search for specific job titles mentioned in skills
        - Company-Specific Search: Search directly on target companies' career pages
        - Location-Based Search: Consider remote, hybrid, and location preferences
        - Experience-Level Filtering: Filter by appropriate experience level

        ## Job Filtering Rules:
        - Experience Level Match: Only include jobs appropriate for candidate's experience level
        - Skill Relevance: Prioritize jobs requiring 70%+ of candidate's skills
        - Active Status: Only include jobs actively accepting applications
        - Application Accessibility: Ensure application process is clearly accessible

        ## Search Execution Instructions
        ### Step 1: Broad Discovery Search
        - Use MCP web search tools with candidate's primary keywords
        - Search major job boards with experience level filters
        - Cast a wide net initially to discover opportunities

        ### Step 2: Targeted Company Search
        - Search career pages of companies in target company profile
        - Use company names + "careers" or "jobs" in search queries
        - Look for new/recent postings on company websites

        ### Step 3: Deep Information Extraction
        - For each promising job found, fetch the complete job posting
        - Navigate to application pages to verify they're active
        - Extract all required information fields

        ### Step 4: Verification & Validation
        - Verify job posting dates are recent (within last 90 days)
        - Confirm application links are functional
        - Check for duplicate postings across platforms

        ## Format: JobPostingList JSON Schema
        - Return a structured JSON object containing:
            - job_postings: List[JobPosting] - List of job postings 
                - List of JobPosting searched for the candidate's profile SCHEMA of JobPosting: {JobPosting.model_json_schema()}
            -   JobPostingList JSON SCHEMA: {JobPostingList.model_json_schema()}
            
        RULES:
        3, IMPORTANT: You must use the candidate's summary (enclosed in <SUMMARY>...</SUMMARY> tags) to filter the job postings relevant to the candidate.
        4. IMPORTANT: You must use the candidate's experience level (enclosed in <EXPERIENCE_LEVEL>...</EXPERIENCE_LEVEL> tags) to filter the job postings relevant to the candidate.
        5. Return only JSON object, no other text.

        Your successis measured by the number of ACTIVE/OPEN jobs found and the quality of the jobs found.
        """

    def get_user_prompt(self) -> str:
        return f"""
         Use the following information to search for the most relevant jobs relevant to the candidate's profile.
        <SUMMARY>
        {self.job_profile.summary}
        </SUMMARY>
        <EXPERIENCE_LEVEL>
        {self.job_profile.experience_level}
        </EXPERIENCE_LEVEL>
        """

    async def get_mcp_server_params(self) -> list:
        """
        This function is used to get the MCP server parameters.
        @return: list - List of dictionaries containing the MCP server parameters
        """
        return [
            {"command": "uvx", "args": ["mcp-server-fetch"]}, 
            {
                "command": "npx",
                "args": ["-y", "@modelcontextprotocol/server-brave-search"],
                "env": self.brave_env,
            }
        ]

    async def get_job_searcher_agent(self, mcp_server_list: list) -> Agent:
        """
        This function is used to create a job searcher agent that can be used to search for jobs based on the candidate's profile.
        @return: Agent - The job searcher agent
        """
        system_prompt = self.get_system_prompt()
        print(f"system_prompt:\n {system_prompt}")
        
        job_searcher_agent = Agent(
            name="JobSearcherAgent",
            instructions=system_prompt,
            mcp_servers=mcp_server_list,
            model=MODEL,
            output_type=JobPostingList
        )

        return job_searcher_agent
    
    async def search_jobs(self) -> JobPostingList:
        """
        This function is used to search for jobs based on the candidate's profile.
        @return: JobPostingList - The list of job postings
        """
        job_postings = None

        user_prompt = self.get_user_prompt()
        print(f"user_prompt:\n {user_prompt}")

        mcp_server_params = await self.get_mcp_server_params()
        mcp_server_list = []
        async with AsyncExitStack() as stack:
            mcp_server_list = [
                await stack.enter_async_context(
                    MCPServerStdio(params, client_session_timeout_seconds=120)
                )
                for params in mcp_server_params
            ]

            job_searcher_agent = await self.get_job_searcher_agent(mcp_server_list)
            
            trace_id = gen_trace_id()
            print(f"Trace@ https://platform.openai.com/api/traces/{trace_id}")
            with trace(trace_id) as tracer:
                job_postings = await Runner.run(job_searcher_agent, user_prompt, max_turns=30)

        return job_postings.final_output if job_postings else None
    

In [28]:
job_postings = await JobSearcherAgent(profiled_resume, MODEL).search_jobs()

print(f"job_postings:\n {job_postings}")

user_prompt:
 
         Use the following information to search for the most relevant jobs relevant to the candidate's profile.
        <SUMMARY>
        - Early-career computational biologist and bioinformatician with hands-on experience in ML-driven genomics and translational oncology research.
- Developed and validated a machine learning model for viral integration site detection in cancer genomes and analyzed large-scale genomic datasets for host–virus interaction insights.
- Trained self-supervised AI models on high-resolution tissue microarrays to extract morphology signatures linked to patient outcomes; strong translational focus.
- Practical wet-lab and analytical chemistry experience (IC, GC, LC-MS/MS) from FDA ORISE fellowship and industry internships, with multiple peer-reviewed publications.
- Skilled in Python, R, Snakemake/Nextflow workflows, containerization (Docker), HPC environments, variant calling, alignment tools, and multi-omics integration.
- Demonstrated ability 

In [30]:
for job_posting in job_postings.job_postings:
    print(job_posting.title)
    print(job_posting.company)
    print(job_posting.link)
    print(job_posting.apply_link)
    print("--------------------------------\n")


Bioinformatics Scientist I
Tempus
https://www.tempus.com/careers/
https://www.tempus.com/careers/
--------------------------------

Associate Bioinformatics Scientist
Guardant Health
https://www.guardanthealth.com/careers/
https://www.guardanthealth.com/careers/
--------------------------------

Bioinformatics Analyst (Early Career)
Illumina
https://www.illumina.com/company/careers.html
https://www.illumina.com/company/careers.html
--------------------------------

Bioinformatics Scientist - Early Career
Foundation Medicine (Roche)
https://www.foundationmedicine.com/careers
https://www.foundationmedicine.com/careers
--------------------------------

Junior Computational Biologist
Broad Institute
https://www.broadinstitute.org/careers
https://www.broadinstitute.org/careers
--------------------------------

Bioinformatics Research Assistant - Computational Genomics
Fred Hutchinson Cancer Center
https://www.fredhutch.org/en/careers.html
https://www.fredhutch.org/en/careers.html
----------